# Passband Ripple
A slight passband ripple effect is applied to simulate analog filtering in RF front ends.

In [ ]:
import torchsig.transforms.functional as F
from torchsig.signals.signal_types import Signal
from torchsig.utils.defaults import default_dataset

import numpy as np
import scipy.signal as signal

%matplotlib inline
import matplotlib.pyplot as plt

Function for generating a test modulated QPSK signal.

In [ ]:
def generate_qpsk_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband QPSK Signal.

    Args:
    num_iq_samples (int, optional): Length of sample. Defaults to 128.
    scale (float, optional): scale normalized signal data. Defaults to 1.0.

    Returns:
        signal: generated Signal.

    """
    dataset = default_dataset(
        signal_generators=["qpsk"],
        num_signals_min=1,
        num_signals_max=1,
        signal_duration_in_samples_min=num_iq_samples,
        signal_duration_in_samples_max=num_iq_samples,
        noise_power_db=0.0,
        start_in_samples=0,
        seed=42,
    )
    signal = dataset.signal_generators[0]()

    # normalize, then scale data
    signal.data = F.normalize(data=signal.data, norm_order=2, flatten=False)
    signal.data = signal.data * scale

    return signal

The passband ripple impairment is applied to a QPSK signal. The impairment is parameterized by the number of taps, the maximum amount of ripple, and an exponential decay rate that controls the rate at which the filter weights decrease to zero.

In [ ]:
# test data
N = 1024
qpsk_data = generate_qpsk_signal(num_iq_samples=N, scale=1.0).data

# apply transform
data_out = F.passband_ripple(
    data=qpsk_data, num_taps=3, max_ripple_db=2, coefficient_decay_rate=2
)

Plots of entire spectrum and zoomed to in-band spectrum responses (frequency normalized). Viewing the zoomed in spectral response will demonstrate a ripple effect, with a slight change from the input signal.

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(2, 1, 1)
fft_size = N
f = np.linspace(-0.5, 0.5 - (1 / fft_size), fft_size)
ax.plot(
    f,
    20 * np.log10(np.abs(np.fft.fftshift(np.fft.fft(qpsk_data)))),
    label="Input Signal",
)
ax.plot(
    f,
    20 * np.log10(np.abs(np.fft.fftshift(np.fft.fft(data_out)))),
    label="Ripple Applied",
)
ax.set_ylabel("Magnitude (dB)")
ax.legend()
ax.grid()
ax.set_ylim([-60, 15])

ax = fig.add_subplot(2, 1, 2)
fft_size = N
f = np.linspace(-0.5, 0.5 - (1 / fft_size), fft_size)
ax.plot(
    f,
    20 * np.log10(np.abs(np.fft.fftshift(np.fft.fft(qpsk_data)))),
    label="Input Signal",
)
ax.plot(
    f,
    20 * np.log10(np.abs(np.fft.fftshift(np.fft.fft(data_out)))),
    alpha=0.5,
    label="Ripple Applied",
)
ax.set_ylabel("Magnitude (dB)")
ax.legend()
ax.grid()
ax.set_ylim([-10, 15])